## Try running knn neibours from 1 to 3 and record testing accuracy

In [ ]:
import ssl 
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from knn_generic import knn_custom,mode
import sys
sys.path.append('../MLPClassification-NNets-Asgmt1/')
from utils.data_loader_CIFAR import load_cifar10_iterators,imshow
from utils.data_loader_Intel import load_Intel 
from sklearn.metrics import classification_report,accuracy_score
# from metric import accuracy
import numpy as np
import matplotlib.pyplot as plt 
import torch
import torchvision.utils
import time 
from tqdm import tqdm 
import seaborn as sns
# define classes in a tuple
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
loaders = load_cifar10_iterators()
train_loader = loaders[0]
test_loader = loaders[1]
val_loader = loaders[2]
# Print Some Data for observation purposes DataLoader Batchsize = 256 no take only 16th row
for images,classes in train_loader:
    print('images.shape:', images.shape)
    print('classes.shape', classes.shape)
    plt.axis('off')
    # without normalization
    imshow(torchvision.utils.make_grid(images,n_row=10))
    # plt.imshow(torchvision.utils.make_grid(images,nrow=32).permute((1,2,0)))
    break
# The same goes for images in other sets

# Trying to record accuracy_score of sklearn.KNeighborsClassifier and custom knn with 1 to 3 Neighbors
k_range = range(1,3)
scores = {}
times = {}
scores_list = []
times_list = []

for k in tqdm(k_range):
    for x_train,y_train in train_loader:
            testiter = iter(test_loader)
            x_test,y_test = next(testiter)
            x_train = np.reshape(x_train, (x_train.shape[0], -1))
            x_test = np.reshape(x_test, (x_test.shape[0], -1))
            start1 = time.time()
            knn = KNeighborsClassifier(n_neighbors=k)
            knn.fit(x_train,y_train)
            y_pred=knn.predict(x_test)
            end1 = time.time()
            print(classification_report(y_test,y_pred))
            times[k] = end1-start1
            times_list.append(times[k])
            scores[k] = accuracy_score(y_test,y_pred)
            scores_list.append(scores[k])
# Classification report 
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
best_k = np.argmax(scores_list)

most_time = np.argmax(times_list)
print("Slowest knn classifier took : {}seconds".format(times_list[most_time]))
#after running the code above, assign to n_neighbors the best performing value
knn_best = KNeighborsClassifier(n_neighbors=best_k)
knn_best.fit(x_train,y_train)
y_pred = knn.predict(x_test)
cmat = confusion_matrix(y_test,y_pred)
plt.figure(0)
disp = ConfusionMatrixDisplay(confusion_matrix=cmat,display_labels=classes)
disp.plot(cmap=plt.cm.Blues)
plt.figure(3)
sns.heatmap(cmat, annot=True)

### Try the same task with NearestCentroidClassifier

In [ ]:
from sklearn.neighbors import NearestCentroid
import sys
sys.path.append('../MLPClassification-NNets-Asgmt1/')
from utils.data_loader_CIFAR import load_cifar10_iterators,imshow
from sklearn.metrics import accuracy_score
from tqdm import tqdm 
import time 
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import seaborn as sns
# define classes in a tuple
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
# define images shape 
# img_rows, img_cols = 32, 32
# input_shape = (img_rows, img_cols, 3)

loaders = load_cifar10_iterators()
train_loader = loaders[0]
test_loader = loaders[1]
val_loader = loaders[2]
# Start timers
start1 = time.time()
# define classes in a tuple
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
ncc_model = NearestCentroid()
for images,labels in train_loader:
    images = np.reshape(images, (images.shape[0], -1))
    ncc_model.fit(images,labels)
stop1 = time.time()
print("Time elapsed {}.s".format(stop1-start1))
testiter = iter(test_loader)
x_test,y_test = next(testiter)
y_pred = ncc_model.predict(np.reshape(x_test,(x_test.shape[0],-1)))
acc = accuracy_score(y_test,y_pred)
print(classification_report(y_test,y_pred))
cmat = confusion_matrix(y_test,y_pred)
plt.figure(0)
disp = ConfusionMatrixDisplay(confusion_matrix=cmat,display_labels=classes)
disp.plot(cmap=plt.cm.Reds)
plt.figure(3)
sns.heatmap(cmat, annot=True)


# Run same classifiers comparison for the Intel Classification Dataset
Note dependencies are loaded in first code cell
# TO BE MODIFIED

In [ ]:
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
import sys
sys.path.append('../MLPClassification-NNets-Asgmt1/')
from utils.data_loader_Intel import load_Intel 
from sklearn.metrics import classification_report,accuracy_score
# from metric import accuracy
import numpy as np
import matplotlib.pyplot as plt 
import time 
from tqdm import tqdm 
import seaborn as sns
X_train,Y_train,X_test,Y_test,X_check,Y_check = load_Intel()
classes = ('buildings','forest','glacier','mountain',
           'sea','street')
from keras_preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1/255,
    zoom_range = 0.1,
    horizontal_flip = True
)

validation_datagen = ImageDataGenerator(
    rescale = 1/255
)
train_generator = train_datagen.flow_from_dataframe(
    dataframe = X_train,
    x_col = "Path",
    y_col = "Label",
    target_size = (150,150),
    batch_size = 1024,
    class_mode = "categorical",
    subset = "training",
    shuffle = True,
    seed = 10
)

validation_generator = validation_datagen.flow_from_dataframe(
    dataframe = X_check,
    x_col = "Path",
    y_col = "Label",
    target_size = (150,150),
    batch_size = 256,
    class_mode = "categorical",
    shuffle = False
)
k_range = range(1,3)
scores_knn = {}
scores_ncc = {}
scores_list_knn = []
scores_list_ncc = []
for k in tqdm(k_range):
    knn_model = KNeighborsClassifier(n_neighbors=k)
    knn_model.fit(X_train,Y_train)
    y_pred = knn_model.predict(X_test)
    print("KNN")
    print(classification_report(Y_test,y_pred))
    scores_knn = accuracy_score(Y_test,y_pred)
    scores_list_knn.append(scores_knn)
    cmat = confusion_matrix(Y_test,y_pred)
    plt.figure(0)
    disp = ConfusionMatrixDisplay(confusion_matrix=cmat,display_labels=classes)
    disp.plot(cmap=plt.cm.Greens)
    plt.figure(3)
    sns.heatmap(cmat, annot=True)
   
    
ncc_model = NearestCentroid() 
ncc_model.fit(X_train,Y_train)   
print("NCC")
y_pred = ncc_model.predict(X_test)
scores_ncc = accuracy_score(Y_test,y_pred)
scores_list_ncc.append(scores_ncc)
print(classification_report(Y_test,y_pred))
plt.figure(0)
disp = ConfusionMatrixDisplay(confusion_matrix=cmat,display_labels=classes)
disp.plot(cmap=plt.cm.Purples)
plt.figure(3)
sns.heatmap(cmat, annot=True)

### Try knn-from scratch

In [ ]:
import sys
sys.path.append('../MLPClassification-NNets-Asgmt1/')
from knn_generic import knn_custom, mode
from sklearn.neighbors import KNeighborsClassifier, NearestCentroid
from metric import accuracy
import time

import matplotlib.pyplot as plt
# from metric import accuracy
import numpy as np
import seaborn as sns
import torch
import torchvision.utils
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
from utils.data_loader_CIFAR import imshow, load_cifar10_iterators
from utils.data_loader_Intel import load_Intel

# define classes in a tuple
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
loaders = load_cifar10_iterators()
train_loader = loaders[0]
test_loader = loaders[1]
val_loader = loaders[2]
# Print Some Data for observation purposes DataLoader Batchsize = 256 no take only 16th row
for images,classes in train_loader:
    print('images.shape:', images.shape)
    print('classes.shape', classes.shape)
    plt.axis('off')
    # without normalization
    imshow(torchvision.utils.make_grid(images,n_row=10))
    # plt.imshow(torchvision.utils.make_grid(images,nrow=32).permute((1,2,0)))
    break


# KNN part
k_range = range(1,3)
scores2= {}
times2 = {}
scores2_list = []
times2_list = []
for k in tqdm(k_range):
     for batch_idx, (x_train, y_train) in enumerate(train_loader):
         start2 = time.time()
         images,labels= next(iter(test_loader))
         _,y_pred = knn_custom(k,images,x_train,mode)
         for label in labels:
              scores[k] = accuracy_score(label,y_pred)
         print(classification_report(labels,y_pred))
         scores2_list.append(scores2[k])
         end2 = time.time()
         times2[k] = end2-start2
         times2_list.append(times2[k])
best_k_generic = np.argmax(scores2_list)
knn_best_generic = knn_custom(best_k_generic,y_test,x_test,mode)
cmat2 = confusion_matrix(y_test,knn_best_generic.y_pred,labels=classes)
disp2 = ConfusionMatrixDisplay(confusion_matrix=cmat2,display_labels=classes)
disp.plot(cmap=plt.cm.Blues)